In [17]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy
import language_check
import re

In [8]:
preds = []

In [9]:
with open('./raw/predictionsRet.json') as json_file:
    sessions = json.load(json_file)

In [10]:
sessions[0][0]

{'wizard_of_wikipedia': 'Elvis Presley',
 'WizardTransformerRanker': {'text': 'Oh baby.... Elvis Presley is truly The King of Rock and Roll!',
  'text_candidates': ['Oh baby.... Elvis Presley is truly The King of Rock and Roll!',
   'I just love listening to the KIng of Rock and Roll - Elvis Presley!',
   'In 1956, Elvis released his first single called "Heartbreak Hotel."  It instantly became a number one hit.  His popularity led him to tv appearances and even movies.  ',
   'Yeah, I love the King of Rock and Roll. Is he your favorite musical artist?',
   'I was 10 when he died, so that may be why.  I visited Graceland with family, and learned about the Memphis Mafia, though!']}}

In [11]:
ind = 0
for session in sessions:
    conv = {}
    firstDiag = session[0]['wizard_of_wikipedia']
    if(len(firstDiag.split("\n")) < 2):
        continue
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [12]:
with open('predictionsRetEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [2]:
preds = []
with open('./raw/predictionsGen.json') as json_file:
    sessions = json.load(json_file)

In [3]:
ind = 0
for session in sessions:
    conv = {}
    firstDiag = session[0]['WizTeacher']
    if(len(firstDiag.split("\n")) < 2):
        continue
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [4]:
with open('predictionsGenEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [146]:

for root, dirs, files in os.walk("./selected/ret"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/ret/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [147]:

for root, dirs, files in os.walk("./selected/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [148]:
tool = language_check.LanguageTool('en-US')
fix_spaces = re.compile(r'\s*([?!.,]+(?:\s+[?!.,]+)*)\s*')

regex = '\\"\s+([^"]+)\s+\\"'

In [149]:
text ="i love the song \" the song \" the song \" the voice \" ."




In [150]:
regex = '\\"\s+([^"]+)\s+\\"'

text = "i love the song \" the song \" the song \" the voice \" ."

subst = ""

# You can manually specify the number of replacements by changing the 4th argument
text = re.sub(regex, '\"'+r'\1'+'\"' , text)

matches = tool.check(text)
text = language_check.correct(text, matches)
text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
text = text.replace("“", "'")
text = text.replace("”", "'")

text

"I love the song 'the song' the song 'the voice'."

In [151]:
for root, dirs, files in os.walk("./final/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        d['actualResponse'] = text
                        
                        newSuggs = []
                        
                        text = d['suggestions'][0]
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        newSuggs.append(text)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [152]:
for root, dirs, files in os.walk("./final/ret"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        
                        subst = ""
                        text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                        matches = tool.check(text)
                        text = language_check.correct(text, matches)
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        text = text.replace("“", "'")
                        text = text.replace("”", "'")
                        text = text.replace(" ' ", "'")
                        
                        d['actualResponse'] = text
                        
                        newSuggs = []
                        
                        for text in d['suggestions']:
                        
                            
                            subst = ""
                            text = re.sub(regex, '\"'+r'\1'+'\"' , text)
                        
                            matches = tool.check(text)
                            text = language_check.correct(text, matches)
                            text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                            text = text.replace("“", "'")
                            text = text.replace("”", "'")
                            text = text.replace(" ' ", "'")
                            
                            newSuggs.append(text)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/ret/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)